
#
# MODULO 07 - EJERCICIO 03-B
# ALEXIS YURI M.


Primero es necesario instalar y configurar PySpark en el notebook de Colab. Para eso se ejecutan las siguientes celdas para instalar las librerías necesarias y crear un contexto de Spark.




In [1]:
# Se instala PySpark y findspark
!pip install pyspark findspark

# Se inicializa findspark para encontrar la instalación de Spark.
import findspark
findspark.init()

# Se importa SparkContext.
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Se crea una sesión de Spark si no existe.
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder.master("local[*]").appName("EjecutarJob").getOrCreate()

sc = spark.sparkContext

print("SparkSession está lista para usar.")

SparkSession está lista para usar.


Paso 1: Se crea un RDD a partir de una lista de números. Se utilizará la API SparkContext.parallelize() para distribuir la lista a través de los nodos de un clúster.


In [2]:
# Se crea una lista de datos simulados.
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Se carga la lista en un RDD.
rdd = sc.parallelize(data)

print(f"El RDD creado es: {rdd}")
print(f"Número de particiones lógicas del RDD: {rdd.getNumPartitions()}")

El RDD creado es: ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289
Número de particiones lógicas del RDD: 2


Paso 2: Se aplican transformaciones.

Las transformaciones en Spark, como filter y map, son lazy. Esto significa que no se ejecutan de inmediato; en su lugar, se construye un plan de ejecución (el DAG).

In [3]:
# Se aplica la transformación "filter".
rdd_filtered = rdd.filter(lambda x: x % 2 == 0)

# Se aplica la transformación "map".
rdd_mapped = rdd_filtered.map(lambda x: x * 10)


Paso 3: Se confirma que aú no se ha ejecutado nada (Lazy Evaluation).


In [4]:
# Se muestra el estado del RDD.

print("El plan de ejecución (DAG) se ha construido, pero aún no se ha ejecutado ninguna operación.")
print(f"Resultado después de aplicar 'filter' al RDD: {rdd_filtered}")
print(f"Resultado después de aplicar 'map' al RDD: {rdd_mapped}")



El plan de ejecución (DAG) se ha construido, pero aún no se ha ejecutado ninguna operación.
Resultado después de aplicar 'filter' al RDD: PythonRDD[1] at RDD at PythonRDD.scala:53
Resultado después de aplicar 'map' al RDD: PythonRDD[2] at RDD at PythonRDD.scala:53



Paso 4: Ejecutar una acción. Spark activa el plan de ejecución para producir un resultado.

In [5]:
# Se ejecuta la acción "mean()".
resultado = rdd_mapped.mean()

print(f"\nSe ha llamado la acción 'mean()'. Esto dispara la ejecución del Job.")
print(f"El valor medio de los datos transformados es: {resultado}")


# Parada ordenada de Spark.
spark.stop()
print("\n")
print("Spark detenido.")




Se ha llamado la acción 'mean()'. Esto dispara la ejecución del Job.
El valor medio de los datos transformados es: 60.0


Spark detenido.


Paso 5:. Visualizar el Job: DAG, Stages y Tasks

Cuando la acción mean() se ejecuta, el SparkContext crea un Job. Este Job se descompone en Stages, y cada Stage se divide en Tasks.

  DAG (Directed Acyclic Graph): Es el plan de ejecución del Job. Representa las dependencias entre las transformaciones y la acción final. Para este ejercicio, el DAG se vería de la siguiente manera:

  parallelize -> filter -> map -> mean

  Stages: El DAG se divide en Stages en función de las operaciones que requieren mover datos entre los nodos del clúster (operaciones de shuffle). En este caso, probablemente haya un solo Stage, ya que filter y map no requieren un shuffle.

  Tasks: Cada Stage se divide en tareas. Una tarea es la unidad de trabajo más pequeña que se envía a un executor (un proceso de trabajo) para que se ejecute en una partición de datos. Si el RDD tiene 4 particiones, habrá 4 tareas para el Stage de filter y 4 para el de map.


------------------------------------------------

Paso 6: Diagrama conceptual del DAG.

graph TD
    A[Lista de números] --> B(RDD)
    B --> C{filter: x % 2 == 0};
    C --> D[map: x * 10];
    D --> E[mean(): acción];
    E --> F[Resultado Final: 60.0];

Explicación:

  Stage 1: Inicia con la lectura de los datos.

  Task 1: Procesa la partición 1 (e.g., [1, 2, 3]) -> Resultado: [2] -> [20]

  Task 2: Procesa la partición 2 (e.g., [4, 5, 6]) -> Resultado: [4, 6] -> [40, 60]

  Task 3: Procesa la partición 3 (e.g., [7, 8, 9]) -> Resultado: [8] -> [80]

  Task 4: Procesa la partición 4 (e.g., [10]) -> Resultado: [10] -> [100]

  Acción mean(): El driver Spark recolecta los resultados parciales de cada tarea ([20, 40, 60, 80, 100]) y calcula el promedio final.

El resultado final, 60.0, es la media de los números pares transformados (2, 4, 6, 8, 10), que al ser multiplicados por 10 son 20, 40, 60, 80 y 100.
